In [65]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [5]:
files = !ls ../../R/visualization/*.csv

In [9]:
data = {file:pd.read_csv(file) for file in files}

In [11]:
data[files[0]]

,Unnamed: 0,Method,Output,Features,Scaling,Overall Score,Batch Correction,PCR batch,Batch ASW,iLISI,...,Bio conservation,NMI cluster/label,ARI cluster/label,Cell type ASW,isolated label F1,isolated label silhouette,CC conservation,HVG conservation,trajectory conservation,cLISI
0,43,Scanorama,embed,HVG,scaled,0.688802,0.585777,0.885319,0.952180,0.309590,...,0.757485,0.627134,0.416023,0.503741,0.167785,0.586023,0.719294,NaN,0.545590,0.826255
1,44,Scanorama,embed,FULL,scaled,0.678498,0.547736,0.835144,0.940894,0.227042,...,0.765672,0.638806,0.381296,0.489245,0.148867,0.474113,0.661380,NaN,0.842511,0.933437
2,151,BBKNN,graph,HVG,unscaled,0.667467,0.871410,NaN,NaN,1.000000,...,0.531504,0.555303,0.468053,NaN,0.078734,NaN,NaN,NaN,0.684339,0.000000
3,212,ComBat,gene,FULL,unscaled,0.641437,0.520101,0.999995,0.913132,0.143482,...,0.722328,0.614375,0.346681,0.470106,0.211682,0.383198,0.867913,0.239344,0.908189,0.954763
4,19,MNN,gene,HVG,scaled,0.625814,0.566140,0.905940,0.932919,0.237516,...,0.665596,0.631690,0.378881,0.496933,0.127805,0.402252,0.762060,0.290062,0.636212,0.932058
5,7,MNN,gene,HVG,unscaled,0.623180,0.391765,0.120337,0.923232,0.165583,...,0.777456,0.640366,0.354824,0.526847,0.209784,0.485932,0.766928,0.343757,0.869389,0.957292
6,67,Scanorama,gene,HVG,scaled,0.622370,0.487988,0.558651,0.876135,0.270893,...,0.711958,0.619437,0.387634,0.503736,0.257360,0.586023,0.738449,0.291859,0.429069,0.892119
7,223,ComBat,gene,HVG,scaled,0.619476,0.544625,1.000000,0.923294,0.204919,...,0.669376,0.607445,0.347841,0.470197,0.121891,0.381516,0.769218,0.253202,0.899172,0.926552
8,20,MNN,gene,FULL,scaled,0.606174,0.481845,0.843837,0.909922,0.179506,...,0.689060,0.625359,0.355528,0.470059,0.223386,0.368323,0.720536,0.086834,0.915112,0.954593
9,224,ComBat,gene,FULL,scaled,0.603394,0.508199,1.000000,0.903349,0.141668,...,0.666857,0.606668,0.321191,0.454931,0.202992,0.365562,0.800576,0.057814,0.922775,0.958361


# Unscaled vs scaled

In [54]:
batch_corr = []
bio_cons = []
overall = []

for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output', 'Features']].values[i]) for i in range(data[file].shape[0])]
    
    for meth in data[file]['method_id'].unique():
        tmpDat = data[file][['Scaling', 'Batch Correction', 'Bio conservation', 'Overall Score']].loc[data[file]['method_id'].isin([meth])]
        if tmpDat.shape[0] == 2:
            overall.append(tmpDat['Overall Score'].loc[tmpDat['Scaling'] == 'unscaled'].values[0] > tmpDat['Overall Score'].loc[tmpDat['Scaling'] == 'scaled'].values[0])
            bio_cons.append(tmpDat['Bio conservation'].loc[tmpDat['Scaling'] == 'unscaled'].values[0] > tmpDat['Bio conservation'].loc[tmpDat['Scaling'] == 'scaled'].values[0])
            batch_corr.append(tmpDat['Batch Correction'].loc[tmpDat['Scaling'] == 'unscaled'].values[0] > tmpDat['Batch Correction'].loc[tmpDat['Scaling'] == 'scaled'].values[0])

In [55]:
np.mean(overall)
np.mean(bio_cons)
np.mean(batch_corr)

0.5050505050505051

0.696969696969697

0.35353535353535354

Unscaled scores tend to have a higher bio conservation score, but a lower batch removal score (70% vs 35% of cases).

In [58]:
len(overall)

99

# HVG vs Full feature

In [62]:
batch_corr = []
bio_cons = []
overall = []

for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output', 'Scaling']].values[i]) for i in range(data[file].shape[0])]
    
    for meth in data[file]['method_id'].unique():
        tmpDat = data[file][['Features', 'Batch Correction', 'Bio conservation', 'Overall Score']].loc[data[file]['method_id'].isin([meth])]
        if tmpDat.shape[0] == 2:
            overall.append(tmpDat['Overall Score'].loc[tmpDat['Features'] == 'HVG'].values[0] > tmpDat['Overall Score'].loc[tmpDat['Features'] == 'FULL'].values[0])
            bio_cons.append(tmpDat['Bio conservation'].loc[tmpDat['Features'] == 'HVG'].values[0] > tmpDat['Bio conservation'].loc[tmpDat['Features'] == 'FULL'].values[0])
            batch_corr.append(tmpDat['Batch Correction'].loc[tmpDat['Features'] == 'HVG'].values[0] > tmpDat['Batch Correction'].loc[tmpDat['Features'] == 'FULL'].values[0])

In [64]:
np.mean(overall)
np.mean(bio_cons)
np.mean(batch_corr)
len(overall)

0.7008547008547008

0.5982905982905983

0.7863247863247863

117

# Method top performer

In [91]:
topMeth = dict()
topMeth_bio = dict()
topMeth_batch = dict()

for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output']].values[i]) for i in range(data[file].shape[0])]
    
    for meth in data[file]['method_id'].unique():
        tmpDat = data[file][['Features', 'Scaling', 'Batch Correction', 'Bio conservation', 'Overall Score']].loc[data[file]['method_id'].isin([meth])]

        if meth not in topMeth:
            topMeth[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}
            topMeth_bio[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}
            topMeth_batch[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}

        tmpDat['preproc'] = ['_'.join(tmpDat[['Features', 'Scaling']].values[i]) for i in range(tmpDat.shape[0])]
        
        topVal = tmpDat.sort_values(by='Overall Score', ascending=False)['preproc'].values[0]
        topMeth[meth][topVal] += 1
        
        topVal = tmpDat.sort_values(by='Bio conservation', ascending=False)['preproc'].values[0]
        topMeth_bio[meth][topVal] += 1
        
        topVal = tmpDat.sort_values(by='Batch Correction', ascending=False)['preproc'].values[0]
        topMeth_batch[meth][topVal] += 1        

In [92]:
topMeth

{'Scanorama_embed': {'HVG_scaled': 6,
  'HVG_unscaled': 0,
  'FULL_scaled': 0,
  'FULL_unscaled': 1},
 'BBKNN_graph': {'HVG_scaled': 0,
  'HVG_unscaled': 6,
  'FULL_scaled': 0,
  'FULL_unscaled': 1},
 'ComBat_gene': {'HVG_scaled': 1,
  'HVG_unscaled': 3,
  'FULL_scaled': 0,
  'FULL_unscaled': 3},
 'MNN_gene': {'HVG_scaled': 2,
  'HVG_unscaled': 3,
  'FULL_scaled': 1,
  'FULL_unscaled': 0},
 'Scanorama_gene': {'HVG_scaled': 5,
  'HVG_unscaled': 2,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'Harmony_embed': {'HVG_scaled': 3,
  'HVG_unscaled': 1,
  'FULL_scaled': 1,
  'FULL_unscaled': 2},
 'Conos_graph': {'HVG_scaled': 0,
  'HVG_unscaled': 3,
  'FULL_scaled': 3,
  'FULL_unscaled': 1},
 'Unintegrated_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 0,
  'FULL_unscaled': 7},
 'scVI_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 7,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'LIGER_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 2,
  'FULL_scaled': 0,
  'FULL_unscaled': 5},
 'Seurat 

# Look at trajectories

In [94]:
data[files[0]].columns

Index(['Unnamed: 0', 'Method', 'Output', 'Features', 'Scaling',
       'Overall Score', 'Batch Correction', 'PCR batch', 'Batch ASW', 'iLISI',
       'graph connectivity', 'kBET', 'Bio conservation', 'NMI cluster/label',
       'ARI cluster/label', 'Cell type ASW', 'isolated label F1',
       'isolated label silhouette', 'CC conservation', 'HVG conservation',
       'trajectory conservation', 'cLISI', 'method_id'],
      dtype='object')

In [146]:
for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output', 'Features','Scaling']].values[i]) for i in range(data[file].shape[0])]
    
    if 'trajectory conservation' not in data[file].columns:
        continue

    print(file)
    print('top performers:')
    data[file].sort_values(by='trajectory conservation', ascending=False)[['method_id', 'trajectory conservation']].reset_index().loc[:8]
    print('bottom performers:')
    data[file].sort_values(by='trajectory conservation', ascending=True)[['method_id', 'trajectory conservation']].reset_index().loc[:8]
    print('\n')

../../R/visualization/immune_cell_hum_mou_summary_scores.csv
top performers:


,index,method_id,trajectory conservation
0,9,ComBat_gene_FULL_scaled,0.922775
1,8,MNN_gene_FULL_scaled,0.915112
2,3,ComBat_gene_FULL_unscaled,0.908189
3,7,ComBat_gene_HVG_scaled,0.899172
4,5,MNN_gene_HVG_unscaled,0.869389
5,30,Harmony_embed_FULL_unscaled,0.849324
6,1,Scanorama_embed_FULL_scaled,0.842511
7,24,BBKNN_graph_FULL_scaled,0.833860
8,29,Conos_graph_HVG_scaled,0.818225


bottom performers:


,index,method_id,trajectory conservation
0,32,scVI_embed_FULL_unscaled,0.297925
1,20,Conos_graph_FULL_unscaled,0.340602
2,16,Conos_graph_HVG_unscaled,0.345803
3,31,LIGER_embed_FULL_unscaled,0.365057
4,12,ComBat_gene_HVG_unscaled,0.400513
5,6,Scanorama_gene_HVG_scaled,0.429069
6,23,Scanorama_gene_FULL_unscaled,0.488941
7,13,Scanorama_gene_HVG_unscaled,0.504163
8,11,Scanorama_embed_HVG_unscaled,0.519381




../../R/visualization/immune_cell_hum_summary_scores.csv
top performers:


,index,method_id,trajectory conservation
0,35,Unintegrated_gene_FULL_unscaled,0.882521
1,6,Scanorama_embed_FULL_unscaled,0.879324
2,5,Conos_graph_FULL_unscaled,0.873814
3,21,Scanorama_gene_FULL_unscaled,0.871638
4,29,MNN_gene_FULL_unscaled,0.866430
5,7,Harmony_embed_HVG_unscaled,0.866057
6,17,ComBat_gene_FULL_unscaled,0.865043
7,2,Scanorama_embed_HVG_unscaled,0.864949
8,1,Conos_graph_HVG_unscaled,0.864343


bottom performers:


,index,method_id,trajectory conservation
0,28,Seurat v3_gene_HVG_unscaled,0.157173
1,31,Seurat v3_gene_HVG_scaled,0.391396
2,19,Scanorama_gene_FULL_scaled,0.558142
3,34,Seurat v3_gene_FULL_unscaled,0.581284
4,36,LIGER_embed_HVG_unscaled,0.586115
5,24,Seurat v3_gene_FULL_scaled,0.646450
6,38,scVI_embed_FULL_unscaled,0.647242
7,37,scVI_embed_HVG_unscaled,0.694126
8,27,Conos_graph_HVG_scaled,0.762621


More complex human, mouse integration requires more batch correction to integrate and thus prefers scaling, while in the simpler case of only human data, unscaled preprocessing performs better. Full generall performs better than HVG.

In general: Combat, MNN, scanorama are good, but Seurat scVI is not. Scanorama gene also does not perform particularly well here. In general full features perform better at trajectory conservation.


# CC conservation

In [106]:
topVer = dict()

for file in data:
    if 'CC conservation' not in data[file].columns:
        continue
    
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output']].values[i]) for i in range(data[file].shape[0])]
    
    for meth in data[file]['method_id'].unique():
        tmpDat = data[file][['Features', 'Scaling', 'CC conservation']].loc[data[file]['method_id'].isin([meth])]

        if tmpDat.shape[0] < 2:
            continue
        
        if meth not in topVer:
            topVer[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}

        tmpDat['preproc'] = ['_'.join(tmpDat[['Features', 'Scaling']].values[i]) for i in range(tmpDat.shape[0])]
        
        topVal = tmpDat.sort_values(by='CC conservation', ascending=False)['preproc'].values[0]
        topVer[meth][topVal] += 1 

In [107]:
topVer

{'Scanorama_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 1,
  'FULL_unscaled': 4},
 'BBKNN_graph': {'HVG_scaled': 0,
  'HVG_unscaled': 5,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'ComBat_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 1,
  'FULL_scaled': 0,
  'FULL_unscaled': 4},
 'MNN_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 1,
  'FULL_unscaled': 3},
 'Scanorama_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 1,
  'FULL_unscaled': 4},
 'Harmony_embed': {'HVG_scaled': 1,
  'HVG_unscaled': 1,
  'FULL_scaled': 2,
  'FULL_unscaled': 1},
 'Conos_graph': {'HVG_scaled': 0,
  'HVG_unscaled': 3,
  'FULL_scaled': 1,
  'FULL_unscaled': 1},
 'scVI_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 4,
  'FULL_scaled': 0,
  'FULL_unscaled': 1},
 'LIGER_embed': {'HVG_scaled': 0,
  'HVG_unscaled': 0,
  'FULL_scaled': 0,
  'FULL_unscaled': 4},
 'Seurat v3_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 1,
  'FULL_scaled': 2,
  'FULL_unscaled': 0},
 'TrVAE_embe

In [114]:
top8Meth = defaultdict(int)
bot8Meth = defaultdict(int)

for file in data:
    data[file] = data[file].loc[~np.isnan(data[file]['Overall Score']),:]
    
    data[file]['method_id'] = ['_'.join(data[file][['Method', 'Output', 'Features','Scaling']].values[i]) for i in range(data[file].shape[0])]
    
    if 'CC conservation' not in data[file].columns:
        continue

    for meth in data[file].sort_values(by='CC conservation', ascending=False)['method_id'].values[:8]:
        top8Meth[meth] += 1

    for meth in data[file].sort_values(by='CC conservation', ascending=True)['method_id'].values[:8]:
        bot8Meth[meth] += 1

In [115]:
top8Meth
bot8Meth

defaultdict(int,
            {'Unintegrated_gene_FULL_unscaled': 5,
             'MNN_gene_FULL_unscaled': 3,
             'ComBat_gene_FULL_unscaled': 4,
             'Scanorama_gene_FULL_unscaled': 4,
             'Scanorama_embed_FULL_unscaled': 4,
             'ComBat_gene_FULL_scaled': 2,
             'ComBat_gene_HVG_scaled': 1,
             'MNN_gene_HVG_unscaled': 1,
             'MNN_gene_HVG_scaled': 2,
             'Harmony_embed_HVG_unscaled': 1,
             'Scanorama_gene_HVG_scaled': 2,
             'Scanorama_embed_FULL_scaled': 2,
             'Scanorama_gene_HVG_unscaled': 2,
             'Scanorama_embed_HVG_unscaled': 1,
             'LIGER_embed_HVG_unscaled': 1,
             'Scanorama_embed_HVG_scaled': 1,
             'MNN_gene_FULL_scaled': 1,
             'Harmony_embed_FULL_scaled': 1,
             'Scanorama_gene_FULL_scaled': 1,
             'Harmony_embed_FULL_unscaled': 1})

defaultdict(int,
            {'LIGER_embed_HVG_unscaled': 3,
             'LIGER_embed_FULL_unscaled': 3,
             'scVI_embed_FULL_unscaled': 4,
             'scVI_embed_HVG_unscaled': 3,
             'Harmony_embed_HVG_scaled': 2,
             'Harmony_embed_HVG_unscaled': 2,
             'Scanorama_embed_FULL_scaled': 1,
             'Scanorama_gene_FULL_scaled': 1,
             'TrVAE_embed_HVG_unscaled': 2,
             'Seurat v3_gene_FULL_unscaled': 2,
             'TrVAE_embed_FULL_unscaled': 2,
             'Seurat v3_gene_HVG_scaled': 3,
             'Seurat v3_gene_HVG_unscaled': 2,
             'Harmony_embed_FULL_unscaled': 1,
             'ComBat_gene_HVG_scaled': 1,
             'ComBat_gene_HVG_unscaled': 2,
             'Scanorama_gene_HVG_scaled': 2,
             'Scanorama_embed_HVG_unscaled': 1,
             'MNN_gene_HVG_unscaled': 1,
             'Scanorama_gene_HVG_unscaled': 1,
             'Scanorama_embed_HVG_scaled': 1})

Full, and especially unscaled data typically conserve more CC variance.

Scanorama, Combat, and MNN perform consistently well

LIGER, scVI, Seurat, and Harmony perform poorly

# HVG conservation

In [122]:
topVer = dict()

for file in data:
    if 'HVG conservation' not in data[file].columns:
        continue
    
    tmp = data[file].loc[~np.isnan(data[file]['HVG conservation']),:].copy()
    
    tmp['method_id'] = ['_'.join(tmp[['Method', 'Output']].values[i]) for i in range(tmp.shape[0])]
    
    for meth in tmp['method_id'].unique():
        tmpDat = tmp[['Features', 'Scaling', 'HVG conservation']].loc[tmp['method_id'].isin([meth])]

        if tmpDat.shape[0] < 2:
            continue
        
        if meth not in topVer:
            topVer[meth] = {'HVG_scaled':0, 'HVG_unscaled':0, 'FULL_scaled':0, 'FULL_unscaled':0}

        tmpDat['preproc'] = ['_'.join(tmpDat[['Features', 'Scaling']].values[i]) for i in range(tmpDat.shape[0])]
        
        topVal = tmpDat.sort_values(by='HVG conservation', ascending=False)['preproc'].values[0]
        topVer[meth][topVal] += 1 

In [123]:
topVer

{'ComBat_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 7,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'MNN_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 6,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'Scanorama_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 7,
  'FULL_scaled': 0,
  'FULL_unscaled': 0},
 'Seurat v3_gene': {'HVG_scaled': 0,
  'HVG_unscaled': 5,
  'FULL_scaled': 0,
  'FULL_unscaled': 0}}

In [133]:
top5Meth = defaultdict(int)
bot5Meth = defaultdict(int)

for file in data:
    tmp = data[file].loc[~np.isnan(data[file]['HVG conservation']),:].copy()
    
    tmp['method_id'] = ['_'.join(tmp[['Method', 'Output', 'Features','Scaling']].values[i]) for i in range(tmp.shape[0])]
    
    if 'HVG conservation' not in tmp.columns:
        continue

    for meth in tmp.sort_values(by='HVG conservation', ascending=False)['method_id'].values[:2]:
        top5Meth[meth] += 1

    for meth in tmp.sort_values(by='HVG conservation', ascending=True)['method_id'].values[:2]:
        bot5Meth[meth] += 1

In [134]:
top5Meth
bot5Meth

defaultdict(int,
            {'Unintegrated_gene_FULL_unscaled': 6,
             'Scanorama_gene_HVG_unscaled': 3,
             'Seurat v3_gene_HVG_unscaled': 3,
             'ComBat_gene_HVG_unscaled': 2})

defaultdict(int,
            {'ComBat_gene_FULL_scaled': 5,
             'MNN_gene_FULL_scaled': 5,
             'ComBat_gene_HVG_scaled': 1,
             'Scanorama_gene_HVG_scaled': 1,
             'Scanorama_gene_FULL_unscaled': 1,
             'Scanorama_gene_FULL_scaled': 1})

Generally performs best on HVG unscaled, worst on full, scaled data



In [140]:
data['../../R/visualization/pancreas_jointnorm_summary_scores.csv'].sort_values(by='Overall Score', ascending=False)[['Method', 'Output', 'Features','Scaling', 'Bio conservation']]

,Method,Output,Features,Scaling,Bio conservation
0,Seurat v3,gene,HVG,unscaled,0.602279
1,Seurat v3,gene,HVG,scaled,0.630728
2,BBKNN,graph,HVG,unscaled,0.649918
3,Seurat v3,gene,FULL,unscaled,0.589893
4,Conos,graph,FULL,scaled,0.755032
5,Conos,graph,HVG,scaled,0.757867
6,Scanorama,embed,HVG,scaled,0.620994
7,Scanorama,gene,HVG,scaled,0.633763
8,Seurat v3,gene,FULL,scaled,0.567241
9,Harmony,embed,FULL,scaled,0.567686
